Я попробовала делать просто эмбеддинги на GraphCodeBERT и OpenAi Codex -- ну так себе работает на претрейне, они не готовы хорошо детектить плагиат. Для их SFT не нашла годных данных, а генерить свои, увы, дороговато. Попробовала исправить это тем, что сначала использовала AST, а потом эмбеддинги, чтобы бвло несколько слоев фильтрации. Замеры, к сожалению, не делала -- только "на глаз" (хотя стоило, да). В итоге мне показалось, что наиболее удачный вариант это построить AST НАД дизассемблированием. По субъективным ощущениям для поставленных целей это наиболее разумный и эффективный вариант

In [ ]:
import os
import dis
import ast
import networkx as nx
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

import plotly.io as pio
pio.renderers.default = "browser"

class BytecodeNode(ast.AST):
    ''' 
    Класс нужен, чтобы байтовый код отдавать в AST как ноды
    '''

    _fields = ['instruction', 'arguments']

    def __init__(self, instruction, arguments):
        self.instruction = instruction
        self.arguments = arguments or []


def disassemble_code(source_code):
    ''' 
    ну тут делаем просто дизассемблинг исходного кода
    '''
    try:
        compiled_code = compile(source_code, '<string>', 'exec')
        return list(dis.get_instructions(compiled_code))
    except Exception as e:
        return e


def bytecode_to_ast(bytecode):
    ''' 
    байткод подаем в AST, чтобы построить дерево
    '''
    root = BytecodeNode('ROOT')
    for instruction in bytecode:
        root.arguments.append(BytecodeNode(instruction.opname, [str(instruction.argval) or ""]))
    return root


def ast_to_text(node, indent):
    ''' 
    теперь то, что получили в AST, превратим в строчку, чтобы можно было превратить ее в вектор
    '''
    result = "  " * indent + f"{node.instruction}("
    result += ", ".join(
        ast_to_text(arg, indent=0).strip() if isinstance(arg, BytecodeNode) else str(arg)
        for arg in node.arguments
    )
    result += ")\n"
    for child in getattr(node, 'arguments', []):
        if isinstance(child, BytecodeNode):
            result += ast_to_text(child, indent + 1)
    return result


def cluster_bytecode_asts(asts, eps = 0.5, min_samples = 2):
    ''' 
    вот тут важно: я строчку AST превращаю в вектор tf-idf-векторайзером предобученным, чтобы можно было посчитать косинусное расстояние
    потом считаю косинусное расстояние и кластеризую дбсканом на косинусных расстояниях 
    '''
    texts = [ast_to_text(ast) for ast in asts]
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(texts)
    similarity_matrix = cosine_similarity(vectors)
    distance_matrix = 1 - similarity_matrix
    distance_matrix[distance_matrix < 0] = 0

    clustering = DBSCAN(metric='precomputed', eps=eps, min_samples=min_samples)
    return clustering.fit_predict(distance_matrix), similarity_matrix


def read_files_from_directory(directory_path: str):
    """Reads Python files from a directory."""
    return {
        filename: open(os.path.join(directory_path, filename), 'r', encoding='utf-8').read()
        for filename in os.listdir(directory_path) if filename.endswith('.py')
    }


def display_clusters_as_graph(filenames, similarity_matrix, clusters):
    ''' 
    ОЧЕНЬ СТРЕМНАЯ ЧАСТЬ ДЛЯ ВИЗУАЛИЗАЦИИ ГРАФА
    '''
    G = nx.Graph()
    for i, file1 in enumerate(filenames):
        G.add_node(file1, cluster=clusters[i])
        for j, file2 in enumerate(filenames):
            if i != j and similarity_matrix[i][j] > 0.5:
                G.add_edge(file1, file2, weight=similarity_matrix[i][j])

    pos = nx.spring_layout(G, seed=42)
    cluster_colors = {c: f"rgb({(hash(c) % 256)}, {(hash(c + 1) % 256)}, {(hash(c + 2) % 256)})" for c in set(clusters)}

    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'), mode='lines')

    node_x, node_y, node_text, node_color = [], [], [], []
    for node, cluster in nx.get_node_attributes(G, 'cluster').items():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(f"{node} (Cluster {cluster})")
        node_color.append(cluster_colors[cluster])

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers', text=node_text,
        marker=dict(colorscale='Viridis', size=10, color=node_color))

    go.Figure(data=[edge_trace, node_trace], layout=go.Layout(
        title="Списывальщики Всея Руси", showlegend=False, hovermode='closest'
    )).show()


def all_proccess(directory_path, eps = 0.5, min_samples = 2):
    ''' 
    Собираем это чудище по лоскуткам

    eps = это эпсилон в дбскане, можно менять ручками
    min_samples = минимальное количество сэмплов в одном кластере

    '''

    files = read_files_from_directory(directory_path)
    if not files:
        print("файлов неть, пожалуйста, положите сюда что-нибудь для тестов")
        return

    filenames = list(files.keys())
    bytecodes = {fname: disassemble_code(code) for fname, code in files.items()}
    asts = {fname: bytecode_to_ast(bytecodes[fname]) for fname in filenames}

    clusters, similarity_matrix = cluster_bytecode_asts(list(asts.values()), eps, min_samples)
    display_clusters_as_graph(filenames, similarity_matrix, clusters)




# Tests

In [ ]:
directory_path = "test_files" 
all_proccess(directory_path)

In [ ]:
directory_path = "tests" 
all_proccess(directory_path)